In [1]:
from __future__ import annotations

import re
from typing import Tuple, Optional

import requests


DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    )
}


def fetch_page(url: str, timeout: float = 15.0) -> Tuple[Optional[str], str]:
    """Fetches a URL and returns (title, text). Best-effort, no external services.

    - Uses requests with a friendly UA and timeout.
    - Tries BeautifulSoup if available for better parsing; otherwise falls back to regex strip.
    """
    resp = requests.get(url, headers=DEFAULT_HEADERS, timeout=timeout)
    resp.raise_for_status()
    html = resp.text

    title = None
    text_content = None

    try:
        from bs4 import BeautifulSoup  # type: ignore

        soup = BeautifulSoup(html, "html.parser")
        title = soup.title.get_text(strip=True) if soup.title else None
        # Remove script/style
        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()
        text_content = soup.get_text(" ", strip=True)
    except Exception:
        # Fallback: naive tag removal
        title_match = re.search(r"<title>(.*?)</title>", html, flags=re.I | re.S)
        if title_match:
            title = re.sub(r"\s+", " ", title_match.group(1)).strip()
        # Remove scripts/styles
        html = re.sub(r"<script[\s\S]*?</script>", " ", html, flags=re.I)
        html = re.sub(r"<style[\s\S]*?</style>", " ", html, flags=re.I)
        # Strip tags
        text_only = re.sub(r"<[^>]+>", " ", html)
        text_content = re.sub(r"\s+", " ", text_only).strip()

    return title, text_content or ""



In [2]:
url = 'https://www.cnbc.com/2025/10/31/cre-companies-ai-goals.html'
fetch_page(url)

("Few CRE companies have achieved their AI goals. Here's why",
 'Few CRE companies have achieved their AI goals. Here\'s why Skip Navigation Markets Pre-Markets U.S. Markets Europe Markets China Markets Asia Markets World Markets Currencies Cryptocurrency Futures & Commodities Bonds Funds & ETFs Business Economy Finance Health & Science Media Real Estate Energy Climate Transportation Industrials Retail Wealth Sports Life Small Business Investing Personal Finance Fintech Financial Advisors Options Action ETF Street Buffett Archive Earnings Trader Talk Tech Cybersecurity AI Enterprise Internet Media Mobile Social Media CNBC Disruptor 50 Tech Guide Politics White House Policy Defense Congress Expanding Opportunity Europe Politics China Politics Asia Politics World Politics Video Latest Video Full Episodes Livestream Top Video Live Audio Europe TV Asia TV CNBC Podcasts CEO Interviews Digital Originals Watchlist Investing Club Trust Portfolio Analysis Trade Alerts Meeting Videos Homestretch

In [4]:
text_content = '''Few CRE companies have achieved their AI goals. Here\'s why Skip Navigation Markets Pre-Markets U.S. Markets Europe Markets China Markets Asia Markets World Markets Currencies Cryptocurrency Futures & Commodities Bonds Funds & ETFs Business Economy Finance Health & Science Media Real Estate Energy Climate Transportation Industrials Retail Wealth Sports Life Small Business Investing Personal Finance Fintech Financial Advisors Options Action ETF Street Buffett Archive Earnings Trader Talk Tech Cybersecurity AI Enterprise Internet Media Mobile Social Media CNBC Disruptor 50 Tech Guide Politics White House Policy Defense Congress Expanding Opportunity Europe Politics China Politics Asia Politics World Politics Video Latest Video Full Episodes Livestream Top Video Live Audio Europe TV Asia TV CNBC Podcasts CEO Interviews Digital Originals Watchlist Investing Club Trust Portfolio Analysis Trade Alerts Meeting Videos Homestretch Jim\'s Columns Education Subscribe PRO Pro News Josh Brown Mike Santoli Calls of the Day My Portfolio Livestream Full Episodes Stock Screener Market Forecast Options Investing Chart Investing Subscribe Livestream Menu Make It select USA INTL Livestream Search quotes, news & videos Livestream Watchlist SIGN IN Create free account Markets Business Investing Tech Politics Video Watchlist Investing Club PRO Livestream Menu Real Estate Housing Construction REITs Rising Risks Newsletter Sign-up CNBC Property Play Just 5% of CRE companies have achieved their AI goals. Here\'s why Published Fri, Oct 31 2025 8:00 AM EDT Updated Fri, Oct 31 2025 8:11 AM EDT Diana Olick @in/dianaolick @DianaOlickCNBC @DianaOlick WATCH LIVE Key Points Real estate companies are moving beyond initial testing and exploration of AI into more targeted applications that aim to redefine value, according to a new survey from JLL. JLL found that 88% of investors, owners and landlords said they have started piloting AI, with most pursuing an average of five use cases simultaneously. Just 5% of respondents said they have achieved all their program goals, while close to half said they have achieved two to three goals. Diminishing perspective of downtown London skyscrapers Chunyip Wong | Istock | Getty Images A version of this article first appeared in the CNBC Property Play newsletter with Diana Olick. Property Play covers new and evolving opportunities for the real estate investor, from individuals to venture capitalists, private equity funds, family offices, institutional investors and large public companies. Sign up to receive future editions, straight to your inbox. The commercial real estate market has been historically slow to modernize, and yet it appears to be accelerating its adoption of artificial intelligence. Companies are moving beyond initial testing and exploration into more targeted applications that aim to redefine value, according to a new survey from JLL. The survey of more than 1,500 senior CRE investor and occupier decision-makers across various industries found that, while still in the early stages, organizations are making AI a priority in their technology budgets. They are also moving from using it just for efficiency to focusing on how it can grow their businesses. JLL found that 88% of investors, owners and landlords said they have started piloting AI, with most pursuing an average of five use cases simultaneously. And more than 90% of occupiers are running corporate real estate AI pilots, according to the report. Compare that with just 5% starting AI pilots two years ago. The adoption is fast, but not entirely easy. Just 5% of respondents said they have achieved all their program goals, while close to half said they have achieved two to three goals. Much of the efforts are still experimental, without much growth. "If you think about commercial real estate, traditionally, it is not a quick technology adopter, and it\'s usually skeptical," said Yao Morin, chief technology officer at JLL. "So the high number of adoptions is actually quite surprising to me. What is not surprising on the flip side is that only 5% actually thinks that they have achieved all the goals. This is pretty aligned with a lot of other industries as well." Get Property Play directly to your inbox CNBC\'s Property Play with Diana Olick covers new and evolving opportunities for the real estate investor, delivered weekly to your inbox. Subscribe here to get access today . The reason they\'re not hitting their goals is because the goal line has moved. Companies have gone beyond just wanting to do certain tasks faster, or so-called operational efficiencies. Now they are tying AI to their revenue goals. For example, some are using it to help them improve their investment risk models, making investment and portfolio decisions based on the output of AI. That will require big changes to the fundamental way they operate. "When you really start moving towards the revenue side, the margin expansion side, then it\'s going to require a lot more than just using a technology," Morin explained. "You can\'t just say, \'Well, I\'m saving you 10% to do this particular thing.\' Companies need to actually rethink their operating model, to rethink how they organize to actually achieve the savings." And so companies are investing heavily in AI, despite economic headwinds. More than half of investors surveyed by JLL have been able to get significant budget growth over the past two years in the space. Their No. 1 spend is on strategic advisory on technology or AI, and most report their budgets have increased solely due to AI. After that, the spending goes to upgrading both cyber- and data-security measures and infrastructure for AI integration. Morin said what she found really surprising is that while most think companies will start using AI for simple tasks, or, low-risk, low-hanging fruit, that was not at all the case. "Our survey showed the opposite. We are getting to a point of sophistication, beyond this initial skeptical phase, where companies are really focusing on the competitive advantage to pressing business problems, using AI to solve instead of [just] those simple low-risk operations." More In CNBC Property Play Why global investment firm Nuveen is betting on this niche real estate subsector Diana Olick Commercial real estate is finally embracing blockchain. Here\'s what investors should know Diana Olick Major real estate developers are fast becoming power brokers Diana Olick Read More Subscribe to CNBC PRO Subscribe to Investing Club Licensing & Reprints CNBC Councils Supply Chain Values CNBC on Peacock Join the CNBC Panel Digital Products News Releases Closed Captioning Corrections About CNBC Internships Site Map Ad Choices Careers Help Contact News Tips Got a confidential news tip? We want to hear from you. Get In Touch CNBC Newsletters Sign up for free newsletters and get more CNBC delivered to your inbox Sign Up Now Get this delivered to your inbox, and more info about our products and services. Advertise With Us Please Contact Us Privacy Policy Your Privacy Choices CA Notice Terms of Service © 2025 CNBC LLC. All Rights Reserved. A Division of NBCUniversal Data is a real-time snapshot *Data is delayed at least 15 minutes.\n      Global Business and Financial News, Stock Quotes, and Market Data\n      and Analysis. Market Data Terms of Use and Disclaimers Data also provided by'''

In [5]:
text_content

'Few CRE companies have achieved their AI goals. Here\'s why Skip Navigation Markets Pre-Markets U.S. Markets Europe Markets China Markets Asia Markets World Markets Currencies Cryptocurrency Futures & Commodities Bonds Funds & ETFs Business Economy Finance Health & Science Media Real Estate Energy Climate Transportation Industrials Retail Wealth Sports Life Small Business Investing Personal Finance Fintech Financial Advisors Options Action ETF Street Buffett Archive Earnings Trader Talk Tech Cybersecurity AI Enterprise Internet Media Mobile Social Media CNBC Disruptor 50 Tech Guide Politics White House Policy Defense Congress Expanding Opportunity Europe Politics China Politics Asia Politics World Politics Video Latest Video Full Episodes Livestream Top Video Live Audio Europe TV Asia TV CNBC Podcasts CEO Interviews Digital Originals Watchlist Investing Club Trust Portfolio Analysis Trade Alerts Meeting Videos Homestretch Jim\'s Columns Education Subscribe PRO Pro News Josh Brown Mike